## system environment

In [ ]:
import sys
print(sys.version)
print(sys.executable)

In [ ]:
import psutil
print("{} GB RAM".format(int(round(psutil.virtual_memory().total/1024/1024/1024))))

In [ ]:
import multiprocessing
print("{} CPU".format(multiprocessing.cpu_count()))

## tensorflow and keras versions, available devices

In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

In [ ]:
for d in tf.config.list_physical_devices():
    print(d)

e.g. (non-GPU)
```
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')
```
or
```
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')
PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
```

## CPU vs. GPU timing

In [ ]:
import numpy as np
import tensorflow as tf
from   time import time

N = 1024
A = np.random.rand(N, N).astype('float32')
B = np.random.rand(N, N).astype('float32')

device_names = ["CPU"]
if tf.config.list_physical_devices("GPU"):
    device_names.append("GPU")

MULTIPLICATIONS = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
REPETITIONS = 3
PRINT_LAST_REPETITION_ONLY = True

N_PATTERN = "{:>16}" 
PER_DEVICE_PATTERN =  "{:>14}"
TIME_PATTERN = "{:.2f} s"

print(N_PATTERN.format("multiplications"), end="")
for device_name in device_names:
    print(PER_DEVICE_PATTERN.format(device_name), end="")
print()

for n in MULTIPLICATIONS:
    for i in range(REPETITIONS):
        infos = [N_PATTERN.format(n)]
        for device_name in device_names:
            t = "n.d."
            if n <= 100 or device_name == "GPU":
                t0 = time()
                with tf.device("/{}:0".format(device_name)):
                    a = tf.Variable(A)
                    b = tf.Variable(B)
                    c = tf.matmul(a, b)
                    for r in range(n - 1):
                        c = tf.matmul(c, b)
                    if device_name != c.device.split(":")[-2]:
                        raise RuntimeError("Unexpected device for '{}': '{}'!".format(device_name, c.device))
                t1 = time()
                t = TIME_PATTERN.format(t1 - t0)
            infos.append(PER_DEVICE_PATTERN.format(t))
        if not PRINT_LAST_REPETITION_ONLY or i == REPETITIONS - 1:
            print("".join(infos))

e.g.
```
 multiplications           CPU           GPU
               1        0.05 s        0.01 s
               2        0.09 s        0.01 s
               5        0.23 s        0.01 s
              10        0.45 s        0.01 s
              20        0.90 s        0.01 s
              50        2.22 s        0.01 s
             100        4.44 s        0.01 s
             200          n.d.        0.23 s
             500          n.d.        0.56 s
            1000          n.d.        1.12 s
            2000          n.d.        2.24 s
            5000          n.d.        5.59 s
           10000          n.d.       11.21 s
```